In [10]:
# General imports
import json
import random
import pandas as pd
from transformers import set_seed


# Intent recognition imports
import torch
torch.manual_seed(42)

# ChatGPT imports
%load_ext autoreload
%autoreload 2
from gpt import GPT
from settings import Settings
from intent_model import IntentModel
from chatbot import ChatbotLogic

# Chat demo
import gradio as gr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
set_seed(SEED)

# Load data 

In [12]:
# Load the data from JSON file
with open("training_data/intents/intent_recognition.json", "r") as f:
    data = json.load(f)

validation_data = []
train_data = []
for datum in data:
    texts = datum["train_questions"]
    label = datum["intent"]
    validation = datum["test_questions"]
    for text in texts:
        train_data.append((text, label))
    for text in validation:
        validation_data.append((text, label))

# Concatenate train and validation data for generating label_map
all_data = train_data + validation_data

# Define the mapping between top-level labels and integers
# Sort the labels before enumerating
label_map = {label: i for i, label in enumerate(sorted(set([data[1] for data in all_data])))}

# Convert the training data labels to integers using the label_map
train_labels = torch.tensor([label_map[data[1]] for data in train_data])
validation_labels = torch.tensor([label_map[data[1]] for data in validation_data])

# Intent recognition model

In [13]:
intent_model = IntentModel(train_data, validation_data, label_map, train_labels, validation_labels)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1, Training Loss: 3.150541067123413
Epoch 2, Training Loss: 3.1332337856292725
Epoch 3, Training Loss: 3.0453290939331055
Epoch 4, Training Loss: 3.022442579269409
Epoch 5, Training Loss: 2.9296350479125977
Epoch 6, Training Loss: 2.8155624866485596
Epoch 7, Training Loss: 2.451505422592163
Epoch 8, Training Loss: 2.217369556427002
Epoch 9, Training Loss: 2.0177042484283447
Epoch 10, Training Loss: 1.9570931196212769
Epoch 11, Training Loss: 1.415897011756897
Epoch 12, Training Loss: 1.108096957206726
Epoch 13, Training Loss: 0.8927435874938965
Epoch 14, Training Loss: 0.6604604721069336
Epoch 15, Training Loss: 0.53182452917099


In [14]:
print(intent_model.get_intent("Can you provide details on the rules pertaining to the age limit for using the company's services?")[0], ": minimum age")
print(intent_model.get_intent("Could you summarize the main provisions of the legal statement?")[0], ": legal_statement_information")
print(intent_model.get_intent("Are you allowed to transfer ownership of my account or subscription to another individual?")[0], ": transfer_legal_statement")

minimum_age : minimum age
legal_statement_information : legal_statement_information
transfer_legal_statement : transfer_legal_statement


In [15]:
# test the intent model
predicted_intent, confidence_scores = intent_model.get_intent("Can you provide details on the rules pertaining to the age limit for using the company's services?")

# Retrieve the intent label and its corresponding confidence score
highest_confidence_intent = predicted_intent

print("Highest confidence intent:", highest_confidence_intent)
print("Highest confidence score:", confidence_scores)
print(len(confidence_scores))
print(len(intent_model.label_map.keys()))

pd.DataFrame({
    'keys': intent_model.label_map.keys(),
    'confidences': confidence_scores
})

Highest confidence intent: minimum_age
Highest confidence score: [0.0010900122579187155, 0.0061378804966807365, 0.0029885212425142527, 0.002026171190664172, 0.006065822672098875, 0.0015116106951609254, 0.0006952295079827309, 0.001502867671661079, 0.0006768866442143917, 0.0009357615490444005, 0.002658248646184802, 0.0032567214220762253, 0.939251184463501, 0.001370432786643505, 0.005653540138155222, 0.0017462322721257806, 0.0053766993805766106, 0.00618339329957962, 0.003425203263759613, 0.003901251358911395, 0.0008488757302984595, 0.002697497606277466]
22
22


,keys,confidences
0,access_personal_data,0.001090
1,cancellation_legal_statement,0.006138
2,change_specific_setting,0.002989
3,contact_privacy_concerns,0.002026
4,cookie_policy,0.006066
5,data_breaches,0.001512
6,data_collection,0.000695
7,data_sharing_opt_out,0.001503
8,data_storage,0.000677
9,delete_personal_information,0.000936


## ChatGPT model

In [16]:
# GPT model here
gpt_model = GPT()

privacy_level = 1 # set to 2 for maximum privacy
give_suggestions = False # set to False to have the bot change the settings for the useres rather than give suggestions
minimum_confidence_score = 0.4

# Load Settings class

In [17]:
settings_model = Settings()

# Test the settings model
# settings_model.update_user_setting("1", "profile_published", False)
settings_model.set_user_calling_card_visibility(user_id = "0", individual_user_id = "2", profile_card_component = "profile_card_component_3_visible", component_state = True)

{'0': {'user_id': '0', 'profile_published': False, 'open_to_connecting': True, 'visibile_in_search': True, 'findable_to': 'logged_on_only', 'individual_user_viewability': [{'user_id': '1', 'profile_card_component_0_visible': False, 'profile_card_component_1_visible': True, 'profile_card_component_2_visible': True, 'profile_card_component_3_visible': True, 'profile_card_component_4_visible': True}, {'user_id': '2', 'profile_card_component_0_visible': True, 'profile_card_component_1_visible': False, 'profile_card_component_2_visible': False, 'profile_card_component_3_visible': True, 'profile_card_component_4_visible': False}]}, '1': {'user_id': '1', 'profile_published': False, 'open_to_connecting': True, 'visibile_in_search': True, 'findable_to': 'everyone'}, '2': {'user_id': '2', 'profile_published': False, 'open_to_connecting': False, 'visibile_in_search': False, 'findable_to': 'logged_on_only'}, '3': {'user_id': '3', 'profile_published': False, 'open_to_connecting': False, 'visibile_i

# Set up gradio chat for demo

In [18]:
theme = gr.themes.Soft(
    primary_hue="orange",
    secondary_hue="orange",
).set(
    body_text_color_dark='*neutral_800',
    background_fill_primary_dark='*neutral_50',
    background_fill_secondary_dark='*neutral_50',
    border_color_accent_dark='*primary_300',
    border_color_primary_dark='*neutral_200',
    color_accent_soft_dark='*primary_50',
    link_text_color_dark='*secondary_600',
    link_text_color_active_dark='*secondary_600',
    link_text_color_hover_dark='*secondary_700',
    link_text_color_visited_dark='*secondary_500',
    block_background_fill='*neutral_100',
    block_background_fill_dark='*neutral_100',
    block_label_background_fill='*primary_400',
    block_label_background_fill_dark='*primary_400',
    block_label_text_color='*neutral_50',
    block_label_text_color_dark='*neutral_50',
    block_title_text_color='*neutral_50',
    block_title_text_color_dark='*neutral_50',
    checkbox_background_color_dark='*background_fill_primary',
    checkbox_background_color_selected='*primary_500',
    checkbox_background_color_selected_dark='*primary_500',
    checkbox_border_color_dark='*neutral_100',
    checkbox_border_color_focus='*primary_300',
    checkbox_border_color_focus_dark='*primary_300',
    checkbox_border_color_hover_dark='*neutral_300',
    checkbox_border_color_selected='*primary_500',
    checkbox_border_color_selected_dark='*primary_500',
    checkbox_border_width_dark='1px',
    checkbox_label_background_fill_selected_dark='*primary_500',
    checkbox_label_text_color_selected_dark='white',
    error_background_fill_dark='#fee2e2',
    error_border_color_dark='#fecaca',
    input_background_fill_dark='white',
    input_background_fill_focus_dark='*secondary_500',
    input_border_color_dark='*neutral_50',
    input_border_color_focus_dark='*secondary_300',
    input_placeholder_color_dark='*neutral_400',
    slider_color_dark='*primary_500',
    stat_background_fill_dark='*primary_300',
    table_border_color_dark='*neutral_300',
    table_even_background_fill_dark='white',
    table_odd_background_fill_dark='*neutral_50',
    button_primary_background_fill_dark='*primary_500',
    button_primary_background_fill_hover_dark='*primary_400',
    button_primary_border_color_dark='*primary_200',
    button_secondary_background_fill_dark='white',
    button_secondary_background_fill_hover_dark='*neutral_100',
    button_secondary_border_color_dark='*neutral_200',
    button_secondary_text_color_dark='*neutral_800'
)

def initialize_chat():
    return ChatbotLogic(data, gpt_model, intent_model, minimum_confidence_score, privacy_level, give_suggestions)

with gr.Blocks(theme=theme, css="assets/chat.css") as demo:
    chatbot_logic = initialize_chat()

    gr.Image("https://iyyu.com/_nuxt/img/navbar_logoW@2x.79eba99.png", interactive=False, tool="image", show_label=False, elem_classes="logo").style(width=200)
    chatbot = gr.Chatbot(chatbot_logic.history)
    msg = gr.Textbox()
    submit = gr.Button("Submit")

    msg.submit(chatbot_logic.user, [msg, chatbot], [msg, chatbot], queue=False).then(
        chatbot_logic.bot, chatbot, chatbot
    )
    submit.click(chatbot_logic.user, [msg, chatbot], [msg, chatbot], queue=False).then(
        chatbot_logic.bot, chatbot, chatbot
    )

demo.launch()

{'0': {'user_id': '0', 'profile_published': False, 'open_to_connecting': True, 'visibile_in_search': True, 'findable_to': 'logged_on_only', 'individual_user_viewability': [{'user_id': '1', 'profile_card_component_0_visible': False, 'profile_card_component_1_visible': True, 'profile_card_component_2_visible': True, 'profile_card_component_3_visible': True, 'profile_card_component_4_visible': True}, {'user_id': '2', 'profile_card_component_0_visible': True, 'profile_card_component_1_visible': False, 'profile_card_component_2_visible': False, 'profile_card_component_3_visible': True, 'profile_card_component_4_visible': False}]}, '1': {'user_id': '1', 'profile_published': False, 'open_to_connecting': True, 'visibile_in_search': True, 'findable_to': 'everyone'}, '2': {'user_id': '2', 'profile_published': False, 'open_to_connecting': False, 'visibile_in_search': False, 'findable_to': 'logged_on_only'}, '3': {'user_id': '3', 'profile_published': False, 'open_to_connecting': False, 'visibile_i

[[None, 'Hello, I am the iYYU privacy bot. I am here to help you with your questions about the company. What can I help you with today?'], ['Can you help me setup my settings?', None]]
2
Prediction: quick_setup  (0.8588468432426453)
3: model is confident!
5: intent is generic
6: ChatGPT should answer.
[[None, 'Hello, I am the iYYU privacy bot. I am here to help you with your questions about the company. What can I help you with today?'], ['Can you help me setup my settings?', None], [None, 'This statement is not found in the provided documentation. However, it seems to be a prompt for the user to set up their profile with a predefined security level. It is important to note that iYYU may have security settings in place to protect user data and privacy.'], ['Can you help me setup my settings?', None]]
2
Prediction: quick_setup  (0.8588468432426453)
3: model is confident!
5: intent is generic
6: ChatGPT should answer.
[[None, 'Hello, I am the iYYU privacy bot. I am here to help you with 

## Imports